In [ ]:
import os
from PIL import Image
import numpy as np
import pickle


In [ ]:
def get_pixele(filename):
    image = Image.open(filename)
    image = image.resize((60,60))
    face_array = np.asarray(image)
    return face_array

In [ ]:
# frontal_faces = []

# path = "./final_dataset/cropped_frontal/"
# for root,dirs,files in os.walk(path):
#     for j,file in enumerate(files):
#         if j%5000==0:
#             print(j)
#         if j%4==0:
#             frontal_faces.append(get_pixele(path+file))

In [ ]:
# frontal_faces = np.array(frontal_faces)
# file = open('frontal_face.pkl', 'wb')
# pickle.dump(frontal_faces, file)


In [ ]:
frontal_faces = np.array([])

file = open('./trained_models/frontal_face.pkl', 'rb')
frontal_faces = pickle.load(file)
frontal_faces.shape

In [ ]:
# side_faces = []

# path = "./final_dataset/cropped_side/"
# for root,dirs,files in os.walk(path):
#     for j,file in enumerate(files):
#         if j%2000==0:
#             print(j)
#         side_faces.append(get_pixele(path+file))

In [ ]:
# side_faces = np.array(side_faces)
# file = open('side_face.pkl', 'wb')
# pickle.dump(side_faces, file)

In [ ]:
side_faces = np.array([])

file = open('./trained_models/side_face.pkl', 'rb')
side_faces = pickle.load(file)
side_faces.shape

In [ ]:
print(frontal_faces.shape)
print(side_faces.shape)

In [ ]:
trainx = np.vstack((frontal_faces,side_faces))
print(trainx.shape)

In [ ]:
labels = np.array([])
zeros = np.zeros((21044,1))
ones = np.ones((16637,1))
labels = np.vstack((zeros,ones))
print(labels.shape)

In [ ]:
from sklearn.utils import shuffle
trainx, labels = shuffle(trainx, labels)

In [ ]:
trainx = trainx/255

In [ ]:
import keras
from keras.layers import *
from keras.models import Sequential
from keras import optimizers

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(60,60,3)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
hist = model.fit(trainx,labels,shuffle=True,batch_size=128,validation_split=0.1,epochs=30)

In [ ]:
model.save("model.h5")

In [ ]:
def get_embedding_test(model, face_pixels):
	face_pixels = face_pixels.astype('float32')
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	samples = np.expand_dims(face_pixels, axis=0)
	yhat = model.predict(samples)
	return yhat[0]

In [ ]:
def get_pixele_test(img_array, required_size=(160, 160)):
    image = Image.fromarray(img_array)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    embedding = get_embedding_test(model, face_array)
    return embedding

In [ ]:
import cv2
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt

In [ ]:
detector = MTCNN()

In [ ]:
cam = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (50, 50) 
fontScale = 1
color = (255, 0, 0) 
thickness = 2

while(True):
    ret,frame = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        continue
    if key_pressed == ord('q'):
        break
    faces = detector.detect_faces(frame)
    for i in range(len(faces)):
        bb = faces[i]['box']
        x, y, w, h = bb
        f_img = frame[y:y+h,x:x+w]
#         embb = get_pixele_test(f_img)
#         print(embb)
#         prediction = model2.predict(embb)
#         frame = cv2.putText(frame, prediction, org, font,fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('yo', f_img)
cv2.waitKey()
cv2.destroyAllWindows()
    